In [1]:
import numpy as np
from ptsa.data.readers import EEGReader,TalReader
from ptsa.data.filters import MorletWaveletFilterCpp,MonopolarToBipolarMapper, MorletWaveletFilter
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneGroupOut
from ptsa.data.TimeSeriesX import TimeSeriesX
import pandas as pd
from scipy.stats import zscore,ttest_rel
%matplotlib inline 
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score,roc_curve
from cluster_helper import cluster
from sklearn.externals import joblib
import cluster_helper.cluster
import xarray

Could not import MorletWaveletFilterCppLegacy (single-core C++ version of MorletWaveletFilter): cannot import name 'MorletWaveletTransform'
You can still use MorletWaveletFilter


In [2]:
def processData(info):
    from ptsa.data.readers import EEGReader
    from ptsa.data.filters import MorletWaveletFilterCpp,MonopolarToBipolarMapper
    import numpy as np
    subject_events = info[0]
    monopolar_channels = info[1]
    bi_pairs = info[2]
    fs = np.array([5.0, 7.0, 11.0,18.0, 27.0, 42.0,80.0])
    wid = 6
    mono_eeg = EEGReader(
        events=subject_events ,
        channels=monopolar_channels, start_time=0.0, end_time=1.6,buffer_time = 1.0).read()
    bi_eeg = MonopolarToBipolarMapper(time_series = mono_eeg, bipolar_pairs = bi_pairs).filter().filtered(
        freq_range=[58.,62.], filt_type="stop")
    mwf = MorletWaveletFilterCpp(time_series=bi_eeg,
                             freqs=fs,
                             output='power',
                             frequency_dim_pos=0,
                             width = wid,
                             verbose=True).filter()[0].remove_buffer(1)
    return mwf.mean('time')

In [3]:
ramFR1_events = np.load('/data/exp_events/ramFR1_events.npy')

subs=  ['R1051J', 'R1060M', 'R1061T', 'R1065J', 'R1137E', 'R1154D', 'R1161E','R1189M', 'R1195E', 'R1236J']
for subjectID in subs:

    tal_path = '/data/tal/'+subjectID+'_talLocs_database_bipol.mat'
    tal_reader = TalReader(filename=tal_path, struct_type = 'bi')
    subject_events = ramFR1_events[(ramFR1_events['subject']==subjectID) 
                                  & (ramFR1_events['type']=='WORD')].view(np.recarray)

    seshs = np.unique(subject_events['session'])
    vaultIdx = np.max(seshs)
    monopolar_channels = tal_reader.get_monopolar_channels()
    bi_pairs = tal_reader.get_bipolar_pairs()
    

    iterable = []
    
    for s in seshs:
        events = subject_events[subject_events['session']==s]
        chans = monopolar_channels
        pairs = bi_pairs
        iterable.append([events,chans,pairs])

    with cluster_helper.cluster.cluster_view(
            scheduler="sge", queue="all.q", num_jobs=4,
            extra_params= {'resources': 'h_vmem=20G'}, cores_per_job=1) as view:
            subResult = view.map(processData, iterable)


    data = xarray.concat(subResult, 'events')
    tmpevs = subResult[0].events.values
    for i in range(1, len(subResult)):
        tmpevs = np.r_[tmpevs, subResult[i].events.values]
    data.events.values = tmpevs.view(np.recarray)

    # features = np.log10(data.stack(features=('frequency','bipolar_pairs')))
    # features

    data.to_hdf('/data/home/jonlevi/ML_Project/iEEG_Power_Files/%s.hdf5' % (subjectID))
    
    

4 Engines running
Sending a shutdown signal to the controller and engines.
4 Engines running
Sending a shutdown signal to the controller and engines.
4 Engines running
Sending a shutdown signal to the controller and engines.
4 Engines running
Sending a shutdown signal to the controller and engines.
4 Engines running
Sending a shutdown signal to the controller and engines.
4 Engines running
Sending a shutdown signal to the controller and engines.
4 Engines running
Sending a shutdown signal to the controller and engines.
4 Engines running
Sending a shutdown signal to the controller and engines.
4 Engines running
Sending a shutdown signal to the controller and engines.
4 Engines running
Sending a shutdown signal to the controller and engines.
